In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %%bash
# set -m
# git clone https://github.com/sachdevkartik/DeepLense.git
# cd DeepLense && git checkout kartik_contribution
# cd ..
# mv DeepLense/Transformers_Classification_DeepLense_Kartik_Sachdev/* .
# rm -rf DeepLense

In [ ]:
# %%bash
# pwd

In [ ]:
!nvidia-smi

In [ ]:
# %%bash
# pip3 install --upgrade -r requirements.txt

In [ ]:
# !pip uninstall numpy

In [ ]:
# !pip install numpy

In [1]:
from __future__ import print_function
import os
import time
import copy
import json
import yaml

import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
import torchvision
from typing import *

In [2]:
from utils.util import *
from config.data_config import DATASET
from utils.dataset import DefaultDatasetSetupSSL
from self_supervised.losses.contrastive_loss import (
    ContrastiveLossEuclidean,
    ContrastiveLossEmbedding,
    SimCLR_Loss,
    NegativeCosineSimilarity,
)
from self_supervised.losses.sym_neg_cos_sim_loss import SymNegCosineSimilarityLoss

from models.modules.head import BYOLProjectionHead, BYOLPredictionHead
from utils.scheduler import cosine_schedule
from torch.utils.data import DataLoader, random_split
from einops.layers.torch import Rearrange


In [3]:
args = {
    "dataset_name": "Model_II",
    "save": "data",
    "num_workers": 8,
    "train_config_path": "self_supervised/config/resnet_byol.yaml",
    "cuda": True,
    "log_dir": "logger"
}

In [4]:
batch_size = 512

dataset_name = args["dataset_name"]
dataset_dir = args["save"]
use_cuda = args["cuda"]
num_workers = args["num_workers"]
train_config_path = args["train_config_path"]
log_dir_base = args["log_dir"]

In [5]:
classes = DATASET[f"{dataset_name}"]["classes"]
num_classes = len(classes)

# Open the YAML file and load its contents
with open(train_config_path, "r") as file:
    train_config = yaml.safe_load(file)

In [6]:
epochs_pretrained = train_config["pretrained"]["num_epochs"]
epochs_finetuned = train_config["finetuned"]["num_epochs"]

learning_rate = train_config["optimizer_config"]["lr"]
margin = train_config["ssl"]["margin"]
num_channels = train_config["channels"]
temperature = train_config["ssl"]["temperature"]
network_type = train_config["network_type"]
image_size = train_config["image_size"]
optimizer_config = train_config["optimizer_config"]

backbone = train_config["backbone"]

make_directories([dataset_dir])
seed_everything(seed=42)

# logging
current_time = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
log_dir = f"{log_dir_base}/{current_time}"
init_logging_handler(log_dir_base, current_time)

# dump config in logger
with open(f"{log_dir}/config.json", "w") as fp:
    json.dump(train_config, fp)

# saving model path location
model_path_pretrained = os.path.join(
    f"{log_dir}/checkpoint",
    f"{network_type}_pretrained_{dataset_name}_{current_time}.pt",
)

model_path_finetune = os.path.join(
    f"{log_dir}/checkpoint",
    f"{network_type}_finetune_{dataset_name}_{current_time}.pt",
)

In [7]:
# setup default dataset
default_dataset_setup = DefaultDatasetSetupSSL(dir=None)
default_dataset_setup.setup(dataset_name=dataset_name)
default_dataset_setup.setup_transforms(image_size=image_size)

>>> Multiple Transforms
>>> Multiple Transforms


In [8]:
# trainset
train_dataset = default_dataset_setup.get_dataset(mode="train")
# default_dataset_setup.visualize_dataset(train_dataset)

Model_II dataset already exists
train data: 89104


In [9]:
# split in train and valid set
split_ratio = 0.25  # 0.25
valid_len = int(split_ratio * len(train_dataset))
train_len = len(train_dataset) - valid_len

train_dataset, val_set = random_split(train_dataset, [train_len, valid_len])

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
)

val_loader = DataLoader(
    dataset=val_set, batch_size=batch_size, shuffle=True, num_workers=num_workers
)

# Load test dataset
# testset = default_dataset_setup.get_dataset(mode="val")
# test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=True)

# size check
sample = next(iter(train_loader))
print("num of classes: ", num_classes)
print(sample[0].shape)

/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anacond

num of classes:  3
torch.Size([512, 1, 224, 224])


In [10]:
class BYOL2(nn.Module):
    def __init__(self, backbone, num_ftrs=512):
        super().__init__()

        self.backbone = backbone
        self.backbone[0] = nn.Conv2d(
            1, 64, kernel_size=7, stride=2, padding=3, bias=False
        )

        self.projection_head = BYOLProjectionHead(num_ftrs, 1024, 256)
        self.prediction_head = BYOLPredictionHead(256, 1024, 256)

        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)

        deactivate_requires_grad(self.backbone_momentum)
        deactivate_requires_grad(self.projection_head_momentum)

    def forward(self, x):
        y = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(y)
        p = self.prediction_head(z)
        return p

    def forward_momentum(self, x):
        y = self.backbone_momentum(x).flatten(start_dim=1)
        z = self.projection_head_momentum(y)
        z = z.detach()
        return z


In [11]:
# Create pretrain model
resnet = torchvision.models.resnet34()
backbone = nn.Sequential(*list(resnet.children())[:-1])

num_ftrs_dict = {
    "resnet18": 512,
    "resnet34": 512,
    "resnet50": 2048,

}
model = BYOL2(backbone, num_ftrs=num_ftrs_dict["resnet34"])

device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# summary(model, input_size=(1, 1, 224, 224), device="cuda")

########################## Pretraining #############################

# optimizer and loss function for pretrain
optimizer_pretrain = torch.optim.SGD(model.parameters(), lr=0.06)

# criterion
criterion = NegativeCosineSimilarity()

In [ ]:
print("Starting Training")
for epoch in range(epochs_pretrained):
    total_loss = 0
    best_loss = float("inf")

    momentum_val = cosine_schedule(epoch, epochs_pretrained, 0.996, 1)
    for batch_idx, (x0, x1, label) in enumerate(train_loader):
        update_momentum(model.backbone, model.backbone_momentum, m=momentum_val)
        update_momentum(
            model.projection_head, model.projection_head_momentum, m=momentum_val
        )
        x0 = x0.to(device)
        x1 = x1.to(device)
        p0 = model(x0)
        z0 = model.forward_momentum(x0)
        p1 = model(x1)
        z1 = model.forward_momentum(x1)
        loss = 0.5 * (criterion(p0, z1) + criterion(p1, z0))
        total_loss += loss.detach()
        loss.backward()
        optimizer_pretrain.step()
        optimizer_pretrain.zero_grad()
        if batch_idx % 10 == 0:
            print(
                f"Epoch [{epoch}/{epochs_pretrained}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item()}"
            )

    if total_loss < best_loss:
        best_loss = total_loss
        best_model = copy.deepcopy(model)
        torch.save(best_model.state_dict(), model_path_pretrained)

    avg_loss = total_loss / len(train_loader)
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

In [ ]:
!nvidia-smi

In [12]:
# load model
pretrained_path = "logger/colab/CrossFormer_pretrained_Model_II_2023-07-31-23-14-06.pt"
model.load_state_dict(torch.load(pretrained_path))
print(model)

BYOL2(
  (backbone): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [13]:
# model.backbone

In [14]:
class FinetuneClassifier(nn.Module):
    def __init__(self, backbone, head):
        super(FinetuneClassifier, self).__init__()

        deactivate_requires_grad(backbone)
        self.backbone = backbone
        self.pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.rearrange = Rearrange("... () () -> ...")
        self.head = head

    def forward(self, x):
        z = self.backbone(x)
        z = self.pool(z)
        z = self.rearrange(z)
        z = self.head(z)
        return z

In [15]:
classification_head = nn.Sequential(
    nn.Linear(num_ftrs_dict["resnet34"], 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Linear(512, num_classes),)

In [16]:
backbone = nn.Sequential(*list(model.backbone.children())[:-1])

finetune_model = FinetuneClassifier(backbone, classification_head)
finetune_model.to(device)
summary(finetune_model, input_size=(1, 1, 224, 224), device="cuda")

Layer (type:depth-idx)                        Output Shape              Param #
FinetuneClassifier                            --                        --
├─Sequential: 1-1                             [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                            [1, 64, 112, 112]         (3,136)
│    └─BatchNorm2d: 2-2                       [1, 64, 112, 112]         (128)
│    └─ReLU: 2-3                              [1, 64, 112, 112]         --
│    └─MaxPool2d: 2-4                         [1, 64, 56, 56]           --
│    └─Sequential: 2-5                        [1, 64, 56, 56]           --
│    │    └─BasicBlock: 3-1                   [1, 64, 56, 56]           (73,984)
│    │    └─BasicBlock: 3-2                   [1, 64, 56, 56]           (73,984)
│    │    └─BasicBlock: 3-3                   [1, 64, 56, 56]           (73,984)
│    └─Sequential: 2-6                        [1, 128, 28, 28]          --
│    │    └─BasicBlock: 3-4                   [1, 128, 28, 28]       

In [17]:
learning_rate = 3e-4
weight_decay =  0.01

finetune_optimizer = optim.AdamW(
    finetune_model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay,
)

finetune_epochs = 10

In [18]:
finetuned_model_path = "logger/colab/CrossFormer_finetuned_II_2023-07-31-23-14-06.pt"
finetune_criterion = nn.CrossEntropyLoss()

In [19]:
def finetune(
    epochs: int,
    model: nn.Module,
    device: Union[int, str],
    train_loader: Any,
    criterion: nn.Module,
    optimizer: nn.Module,
    saved_model_path: str,
    valid_loader: Any,
):
    best_loss = float("inf")
    all_val_loss = []
    all_val_accuracy = []

    # Training loop
    for epoch in range(epochs):
        epoch_loss = 0.0
        model.train()

        for batch_idx, (img1, _, label) in enumerate(train_loader):
            img1 = img1.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            output = model(img1)

            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            epoch_loss += loss

            if batch_idx % 10 == 0:
                print(
                    f"Epoch [{epoch}/{epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item()}"
                )

        epoch_loss = epoch_loss / len(train_loader)

        if epoch_loss < best_loss:
            best_loss = epoch_loss
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), saved_model_path)
            print("====== Model saved ======")

        with torch.no_grad():
            print("====== Eval started ======")
            model.eval()
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            for _, (data, _, label) in enumerate(valid_loader):
                data = data.to(device)
                label = label.to(device)

                val_output = model(data)
                val_loss = criterion(val_output, label)

                acc = (val_output.argmax(dim=1) == label).float().mean()
                epoch_val_accuracy += acc
                epoch_val_loss += val_loss

            epoch_val_accuracy = epoch_val_accuracy / len(valid_loader)
            epoch_val_loss = epoch_val_loss / len(valid_loader)
            all_val_loss.append(epoch_val_loss)

        all_val_accuracy.append(epoch_val_accuracy.item() * 100)

        print(
            f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f} \n"
        )

In [20]:
# Training loop
finetune(
    finetune_epochs,
    finetune_model,
    device,
    train_loader,
    finetune_criterion,
    finetune_optimizer,
    finetuned_model_path,
    valid_loader=val_loader,
)

/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anacond

Epoch [0/10], Batch [0/131], Loss: 1.156130075454712
Epoch [0/10], Batch [10/131], Loss: 1.0891685485839844
Epoch [0/10], Batch [20/131], Loss: 1.0873832702636719
Epoch [0/10], Batch [30/131], Loss: 1.0675220489501953
Epoch [0/10], Batch [40/131], Loss: 1.0510790348052979
Epoch [0/10], Batch [50/131], Loss: 1.0583583116531372
Epoch [0/10], Batch [60/131], Loss: 1.0402311086654663
Epoch [0/10], Batch [70/131], Loss: 1.0362662076950073
Epoch [0/10], Batch [80/131], Loss: 1.0036343336105347
Epoch [0/10], Batch [90/131], Loss: 1.0043225288391113
Epoch [0/10], Batch [100/131], Loss: 0.9899460077285767
Epoch [0/10], Batch [110/131], Loss: 1.022749900817871
Epoch [0/10], Batch [120/131], Loss: 0.9831846952438354
Epoch [0/10], Batch [130/131], Loss: 0.9697375893592834
====== Model saved ======
====== Eval started ======


/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anacond

Epoch : 1 - loss : 1.0382 - val_loss : 0.9736 - val_acc: 0.5581 



/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anacond

...

Epoch [5/10], Batch [0/131], Loss: 0.8453021049499512
Epoch [5/10], Batch [10/131], Loss: 0.8118219971656799
Epoch [5/10], Batch [20/131], Loss: 0.8211382627487183
Epoch [5/10], Batch [30/131], Loss: 0.7973555326461792
Epoch [5/10], Batch [40/131], Loss: 0.8319476842880249
Epoch [5/10], Batch [50/131], Loss: 0.7941102981567383
Epoch [5/10], Batch [60/131], Loss: 0.8466867804527283
Epoch [5/10], Batch [70/131], Loss: 0.7994270920753479
Epoch [5/10], Batch [80/131], Loss: 0.8073349595069885
Epoch [5/10], Batch [90/131], Loss: 0.7666801810264587
Epoch [5/10], Batch [100/131], Loss: 0.7723305821418762
Epoch [5/10], Batch [110/131], Loss: 0.7985684871673584
Epoch [5/10], Batch [120/131], Loss: 0.7708105444908142
Epoch [5/10], Batch [130/131], Loss: 0.851091206073761
====== Model saved ======
====== Eval started ======


/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anacond

Epoch : 6 - loss : 0.8086 - val_loss : 0.8141 - val_acc: 0.6016 



/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anaconda3/envs/dlvr/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2019: UserWarning: The image is already gray.
  warnings.warn("The image is already gray.")
/home/kartik/anacond

Epoch [6/10], Batch [0/131], Loss: 0.8009103536605835
Epoch [6/10], Batch [10/131], Loss: 0.814725935459137
Epoch [6/10], Batch [20/131], Loss: 0.7795451879501343
Epoch [6/10], Batch [30/131], Loss: 0.8139488697052002
Epoch [6/10], Batch [40/131], Loss: 0.8131596446037292
Epoch [6/10], Batch [50/131], Loss: 0.8241100907325745
Epoch [6/10], Batch [60/131], Loss: 0.7481332421302795
Epoch [6/10], Batch [70/131], Loss: 0.812390148639679
Epoch [6/10], Batch [80/131], Loss: 0.8005272746086121
Epoch [6/10], Batch [90/131], Loss: 0.8238431215286255
Epoch [6/10], Batch [100/131], Loss: 0.8032150864601135
Epoch [6/10], Batch [110/131], Loss: 0.8012538552284241


# New Section